In [24]:
import datetime
import os
from functools import lru_cache
from BaseHaas import Bot
import dateutil.relativedelta
import ipywidgets as widgets
import pandas as pd
from haasomeapi.enums.EnumMadHatterIndicators import EnumMadHatterIndicators
from haasomeapi.enums.EnumMadHatterSafeties import EnumMadHatterSafeties
from haasomeapi.HaasomeClient import HaasomeClient
from ipywidgets import interact
from haasomeapi.enums.EnumErrorCode import EnumErrorCode
from ratelimit import limits, sleep_and_retry
import configserver
from botsellector import BotSellector

def calculate_ticks(start_date,end_date):
    diff = end_date-start_date
    secs = diff.total_seconds()/60
    return int(secs)
ticks = interact(calculate_ticks,
        start_date=widgets.DatePicker(value=(datetime.date.today() + dateutil.relativedelta.relativedelta(days=-1))),
        end_date=widgets.DatePicker(value=datetime.date.today()))

interactive(children=(DatePicker(value=datetime.date(2020, 5, 13), description='start_date'), DatePicker(value…

Changes in Bot configuration and backtesting are like family: one cannot be whole without the other one. Yet up until now, we didn't have them working in parallel. From now on things have changed: backtesting follows bot parameter changes automatically whenever this script is enabled.

In [26]:
class InteractiveBT(Bot):
    def __init__(self):
        Bot.__init__(self)
        
    @sleep_and_retry
    @limits(calls=3, period=2)
    def return_botlist(self):
        bl = self.c().customBotApi.get_all_custom_bots().result
        botlist = [x for x in bl if x.botType == 15]
        # print(botlist)
        return botlist
    

    def return_edited_bot(self):
        botlist = self.return_botlist()
        while True:
            botlist2 = self.return_botlist()
            lists = zip(botlist,botlist2)
            for x in lists:
                c= self.compare_indicators(x[0],x[1])
                if c == False:
                    return x[1]
    @sleep_and_retry
    @limits(calls=4, period=3)
    def monitor_bot(self,bot, ticks):
        botlist = self.return_botlist()
        for b in botlist:
            if b.guid == bot.guid:
                c = self.compare_indicators(bot,b)
                if c == True:
                    pass
                elif c == False:
                    bot = self.bt_mh_on_update(b, ticks)
                return bot
           
    def compare_indicators(self, bot, bot1):
            # print(bot.rsi, '\n',bot1.rsi)
        rsi = bot.rsi.items() == bot1.rsi.items()
        bbands = bot.bBands.items() == bot1.bBands.items()
        macd = bot.macd.items() == bot1.macd.items()
        interval = bot.interval == bot1.interval
        if rsi == True and bbands == True and macd == True and interval == True:
            return True
        else:
            # print('bot not alike')
            return False



    @sleep_and_retry
    @limits(calls=3, period=2)
    def bt_mh_on_update(self, bot, ticks):

        bt = self.c().customBotApi.backtest_custom_bot(
            bot.guid,
            int(ticks)
        )
        if bt.errorCode != EnumErrorCode.SUCCESS:
            print("bt", bt.errorCode, bt.errorMessage)
        else:
            # print(bt.result.roi)
            # print(bt.errorCode, bt.errorMessage)
            return bt.result
        




1

Here we are creating interactive instance of interactiveBT class.

TO STOP BACKTESTING, APPLY 1 1 1 to MACD parameters.

In [31]:
ibt = InteractiveBT()
bot = ibt.return_edited_bot()
results = []
while True:
    bot = ibt.monitor_bot(bot, ticks.widget.result)
    results.append(bot)
    if bot.macd['MacdFast'] == 1 and bot.macd['MacdSlow'] == 1 and bot.macd['MacdSign'] == 1:
        break

In [32]:
sortedresults = sorted(results,key=lambda x: x.roi, reverse=True)
df = pd.DataFrame([{'ROI':b.roi,'Bot':b} for b in results]) 
# print(df)
# print(df.iloc[0][1])
df.sort_values(by='ROI',inplace=True)


        ROI                                                Bot
0   -0.1028  <haasomeapi.dataobjects.custombots.BaseCustomB...
1   -0.1028  <haasomeapi.dataobjects.custombots.BaseCustomB...
2   -0.1028  <haasomeapi.dataobjects.custombots.BaseCustomB...
3   -0.1028  <haasomeapi.dataobjects.custombots.BaseCustomB...
4   -0.1028  <haasomeapi.dataobjects.custombots.BaseCustomB...
..      ...                                                ...
128 -3.4327  <haasomeapi.dataobjects.custombots.BaseCustomB...
129 -3.4327  <haasomeapi.dataobjects.custombots.BaseCustomB...
130 -3.4327  <haasomeapi.dataobjects.custombots.BaseCustomB...
131 -3.4327  <haasomeapi.dataobjects.custombots.BaseCustomB...
132 -0.1016  <haasomeapi.dataobjects.custombots.BaseCustomB...

[133 rows x 2 columns]


In [34]:

class BotDB:
    def __init__(self):
        self.c = self.cnt()

    def cnt(self):
        ip, secret = configserver.validateserverdata()
        haasomeClient = HaasomeClient(ip, secret)
        return haasomeClient

    def csv_to_sellectionbox(self):
        files = self.get_csv_files()

        return files

    def get_csv_files(self):
        files =[]
        for file in os.listdir('./'):
            if file.endswith(".csv"):
                files.append(os.path.join('./', file))


        return files

    def select_from_list(self,files):
        for i, file in enumerate(files):
                    print(i, file)
        userinput = input('Type file number to select it:  ')
        self.db_file = files[int(userinput)]
        return files[int(userinput)]




    def get_mh_bots(self):
        all_bots = BotSellector().get_all_custom_bots() #getting all bots here
        all_mh_bots = [x for x in all_bots if x.botType == 15] #sorting them to only Mad Hatter Bot(bot type 15 )
        opts= [[x.name,x] for x in all_mh_bots] #making botlist with names
        return opts


    def setup_bot(self,bot,config): #calling it setup_bot. It checks each parameter against new config.
            if bot.bBands["Length"] != config.bBands['Length']: #if params differ - applies new one.
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter( #this way less api calls is being made
                bot.guid,
                EnumMadHatterIndicators.BBANDS,
                0,
                    config.bBands['Length']
            )

            if bot.bBands["Devup"] != config.bBands['Devup']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                    bot.guid,
                    EnumMadHatterIndicators.BBANDS,
                    1,
                    config.bBands['Devup'],
                )


            if bot.bBands["Devdn"] != config.bBands['Devdn']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.BBANDS,
                2,
                config.bBands['Devdn'],
                )


            if bot.bBands["MaType"] != config.bBands['MaType']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.BBANDS,
                3,
                config.bBands['MaType'],
                )


            if bot.bBands["AllowMidSell"] != config.bBands['AllowMidSell']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.BBANDS,
                5,
                config.bBands['AllowMidSell'],
                )


            if bot.bBands["RequireFcc"] != config.bBands['RequireFcc']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.BBANDS,
                6,
                config.bBands['RequireFcc'],
                )


            if bot.rsi["RsiLength"] != config.rsi['RsiLength']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.RSI,
                0,
                config.rsi['RsiLength'],
                )


            if bot.rsi["RsiOverbought"] != config.rsi['RsiOverbought']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.RSI,
                1,
                config.rsi['RsiOverbought'],
                )


            if bot.rsi["RsiOversold"] != config.rsi['RsiOversold']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.RSI,
                2,
                config.rsi['RsiOversold'],           )


            if bot.macd["MacdFast"] != config.macd['MacdFast']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.MACD,
                0,
                config.macd['MacdFast'],
                )

            if bot.macd["MacdSlow"] != config.macd['MacdSlow']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.MACD,
                1,
                config.macd['MacdSlow'],
                )


            if bot.macd["MacdSign"] != config.macd['MacdSign']:
                do = self.c.customBotApi.set_mad_hatter_indicator_parameter(
                bot.guid,
                EnumMadHatterIndicators.MACD,
                2,
                config.macd['MacdSign'],
                )

                                                                        #Indicator parameters have been set


            if bot.interval != config.interval:
                setup_bot = self.c.customBotApi.setup_mad_hatter_bot( # This code sets time interval as main goalj
                botName = bot.name,
                botGuid=bot.guid,
                accountGuid=bot.accountId,
                primaryCoin=bot.priceMarket.primaryCurrency,
                secondaryCoin=bot.priceMarket.secondaryCurrency,
                contractName=bot.priceMarket.contractName,
                leverage=bot.leverage,
                templateGuid=bot.customTemplate,
                position=bot.coinPosition,
                fee=bot.currentFeePercentage,
                tradeAmountType=bot.amountType,
                tradeAmount=bot.currentTradeAmount,
                useconsensus=bot.useTwoSignals,
                disableAfterStopLoss=bot.disableAfterStopLoss,
                interval=config.interval,
                includeIncompleteInterval=bot.includeIncompleteInterval,
                mappedBuySignal=bot.mappedBuySignal,
                mappedSellSignal=bot.mappedSellSignal,).result


            print(bot.name,' Has been configured')


    def calculate_ticks(start_date,end_date):
        diff = end_date-start_date
        secs = diff.total_seconds()/60
        return int(secs)
    @lru_cache(maxsize=None)
    def bt_bot(self,bot,depth):
        bt = self.c.customBotApi.backtest_custom_bot(
  
                    bot.guid,
                    int(depth))
 
        print(bt.result.roi)
        return bt.result


    


bot = BotDB().bt_bot(df.iloc[0][1],ticks.widget.result)
bd = BotDB()
setup = bd.setup_bot(bot,df.iloc[0][1])

In [ ]:
Now, int he next cell is where you can select bot configuration by its roi and apply it to the bot

In [36]:

    

botsellector = widgets.Dropdown(#Making dropdown out of it
    options= [[x.roi,x] for x in results],
    value=[x for x in results][0],
    description='Botlist',
    disabled=False,
)
display(botsellector) #displaying dropdown list here

Dropdown(description='Botlist', options=([-0.1028, <haasomeapi.dataobjects.custombots.BaseCustomBot.BaseCustom…

In [37]:
bot = BotDB().bt_bot(botsellector.value,1)
setup = bd.setup_bot(bot,botsellector.value)
bot = BotDB().bt_bot(botsellector.value,ticks.widget.result)

0.0
4  Has been configured
-4.8162
